In [2]:
# Given prior probabilities and test result likelihoods
prior1 = 0.001  # p(X = 1)
FN = 0.01  # p(Y = 0 | X = 1)
FP = 0.1  # p(Y = 1 | X = 0)

# Calculate the complementary probabilities
prior0 = 1 - prior1 # p(X = 0)
TP = 1 - FN # p(Y = 1 | X = 1)
TN = 1 - FP  # p(Y = 0 | X = 0)

## Q1

In [3]:
# 1(a) calculate p(X = 1 | Y = 1) using Bayes Rule
normalization_factor_1a = prior0 * FP + prior1 * TP # p(Y = 1)
posterior_1a = TP * prior1 / normalization_factor_1a
posterior_1a

0.009812667261373772

In [4]:
# 1(b) calculate p(X = 1 | Y = 0) using Bayes Rule
normalization_factor_1b = prior0 * TN + prior1 * FN # p(Y = 0)
posterior_1b = FN * prior1 / normalization_factor_1b
posterior_1b 

1.112210964175685e-05

## Q3

In [5]:
# Given prior probabilities and test result likelihoods
x1 = 0.001  # p(X = 1)
y0_x1 = 0.01  # p(Y = 0 | X = 1)
y1_x0 = 0.1  # p(Y = 1 | X = 0)

# Calculate the complementary probabilities
x0 = 1 - x1 # p(X = 0)
y1_x1 = 1 - FN # p(Y = 1 | X = 1)
y0_x0 = 1 - FP  # p(Y = 0 | X = 0)

x1_y1 = posterior_1a
x0_y1 = 1 - x1_y1 # p(X = 0 | Y = 1)

x1_y0 = posterior_1b
x0_y0 = 1 - x1_y0 # p(X = 0 | Y = 0)

In [6]:
# 3(a) calculate p(X = 1 | Y = 0) using Bayesian estimator
likelihood = y1_x1
prior = x1_y1
evidence = y1_x0 * (1-prior) + y1_x1 * prior
posterior = likelihood * prior / evidence
posterior

0.08934285011075559

In [20]:
#3(b)
def recursive_update_posterior(posterior, TP, FP, count=0):
    """
    Recursive function to update the posterior probability based on new positive test results (Yn = 1).
    
    :param posterior: The current posterior probability of having the disease.
    :param TP: The true positive rate of the test.
    :param FP: The false positive rate of the test.
    :return: The updated posterior probability.
    """
    # calculate evidence
    evidence = TP * posterior + FP * (1 - posterior) # posterior (from last iter) here means the prior
    # update the posterior probability
    updated_posterior = (TP * posterior) / evidence # real posterior of current iter
     
    # num of test iter
    count += 1
    
    # updated posterior is greater than 0.5, return the value and number of tests
    if updated_posterior > 0.5:
        return updated_posterior, count
    else:
        # call the function recursively
        return recursive_update_posterior(updated_posterior, TP, FP, count)

# initial posterior
posterior_i = x1_y1
posterior_final, count = recursive_update_posterior(posterior_i, y1_x1, y1_x0, count=1)
posterior_final, count

(0.9057987969233379, 4)

## Q4

In [4]:
import numpy as np

# possibily of test results
TN = 0.9
FN = 0.01
TP = 0.99
FP = 0.1

# Initial belief
pi_0 = np.array([1,0])

# transition matrix (no change)
T = np.array([
    [0.999, 0.05],
    [0.001, 0.95]
])

# likelihood
M_neg = np.array([[TN,0],
                 [0,FN]])
M_pos = np.array([[FP,0],
                 [0,TP]])

def update_belief(pi_prev, test_result, M_neg, M_pos, T):
    if test_result:
        M = M_pos # test positive
    else:
        M = M_neg # test negative
    pi_cur = M @ T @ pi_prev
    pi_cur_normalized = pi_cur / np.sum(pi_cur)
    return pi_cur_normalized

# iterate through 98 negative tests
pi_cur = pi_0 # belief = initial belief
for day in range(100):
    # Negative tests for day 1 to day 98, positive test on day 99
    pi_cur = update_belief(pi_cur, day == 99, M_neg, M_pos, T)
    if day % 10 ==0 or day == 99:
        print(day, pi_cur)

# probability of being infected on day 99
print("the final probability is",pi_cur[1])

0 [9.99988878e-01 1.11221096e-05]
10 [9.99988759e-01 1.12408744e-05]
20 [9.99988759e-01 1.12408744e-05]
30 [9.99988759e-01 1.12408744e-05]
40 [9.99988759e-01 1.12408744e-05]
50 [9.99988759e-01 1.12408744e-05]
60 [9.99988759e-01 1.12408744e-05]
70 [9.99988759e-01 1.12408744e-05]
80 [9.99988759e-01 1.12408744e-05]
90 [9.99988759e-01 1.12408744e-05]
99 [0.99008359 0.00991641]
the final probability is 0.009916411596790707
